# Variational Quantum Eigensolver Tutorial

So you want to find the lowest eigenvalue (and eigenvector) of a very large matrix. Maybe that matrix represents the energy operator of some quantum system whose ground state you're interested in. Maybe you have other motives. 

Since the matrix is very large, it's going to be inconvenient to directly solve for its eigenvalues and eigenvectors. On the other hand, if we actually had a quantum system, we could evaluate expectation values like $\langle \psi \mid H \mid \psi \rangle$. Could we use that to help us? 

Suppose we had some anstaz for the eigenvector that takes some parameters, for example, an angle $\theta$: $\alpha(\theta)$. We could search over $\langle \alpha(\theta) \mid H \mid \alpha(\theta) \rangle$ until we find a value of $\theta$ that minimizes the value of the operator, and then we've solved our problem.

In order to do this, however, we have to figure out how to actually measure $\langle \psi \mid H \mid \psi \rangle$ on our quantum computer. All we have are $n$ qubits, the ability to implement unitary gates, and the ability to measure each of the qubits along the $Z$ axis, obtaining a bit: $0$ or $1$. So what do we do?

<hr>

For example, suppose we have $H = \begin{pmatrix} 4 & -2i \\ 2i & 2 \end{pmatrix}$.

Let's stare at the Pauli matrices:

$I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}, X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} $

After a while, we might realize that $H$ can be written $3 + 2Y + Z$.

$\begin{pmatrix} 4 & -2i \\ 2i & 2 \end{pmatrix} = 3\begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} + 2\begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix} + \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} $

By the laws of expectation values, it follows that:

$\langle H \rangle = 3+ 2\langle Y \rangle + \langle Z \rangle$

In other words, instead of measuring $\langle H \rangle$ directly, we could measure $\langle Y \rangle$ and $\langle Z \rangle$ separately (on separate copies of the system), and add up the results, appropriately weighted, to obtain $\langle H \rangle$. In other words, we can split our expectation value into a weighted sum of expectation values of Pauli's.

Indeed, $I, X, Y, Z$ form a basis for $2 \times 2$ Hermitian operators, and so we can decompose any $2\times 2$ Hermitian operator into a sum as we did above: 

$H = \frac{tr(H)}{2}I + \frac{tr(XH)}{2}X + \frac{tr(YH)}{2}Y + \frac{tr(ZH)}{2}Z$


In [ ]:
import numpy as np
import qutip as qt

IXYZ = {"I": qt.identity(2),\
        "X": qt.sigmax(),\
        "Y": qt.sigmay(),\
        "Z": qt.sigmaz()}

def op_pauli(O, basis):
    return dict([(s, (o.dag()*O).tr()/np.sqrt(len(basis)))\
                    for s, o in basis.items()])

def pauli_op(P, basis):
    return sum([P[s]*o for s, o in basis.items()])

def print_pauli(P):
    for s, v in P.items():
        if not np.isclose(v, 0):
            print("%s: %.2f" % (s, v.real))

H = qt.Qobj(np.array([[4, -2j],\
                      [2j, 2]]))
H_pauli = op_pauli(H, IXYZ)
H2 = pauli_op(H_pauli, IXYZ)

print(H)
print_pauli(H_pauli)
print(H2)

We can check that $\langle H \rangle = 3+ 2\langle Y \rangle + \langle Z \rangle$.

In [ ]:
def expect(H, psi, basis):
    H_pauli = op_pauli(H, basis)
    return sum([v*(psi.dag()*basis[s]*psi\
                       if s.count("I") != len(s)\
                   else 1)\
                if not np.isclose(v, 0) else 0
                    for s, v in H_pauli.items()])[0,0]
    
psi = qt.rand_ket(2)
print((psi.dag()*H*psi)[0,0])
print(expect(H, psi, IXYZ))

<hr>

So we've reduced the problem of measuring $\langle H \rangle$ to that of measuring $\langle X \rangle$/$\langle Y \rangle$/$\langle Z \rangle$'s separately (the constant term doesn't require any actual measurement). $\langle Z \rangle$ presents no problem for our quantum computer: we simply prepare some large number $N$ of identical copies of the initial qubit, and measure them: we get $\uparrow$ or $\downarrow$ each time. Over the course of the experiments, we accumulate counts, the number $N_{\uparrow}$ and the number $N_{\downarrow}$, which we divide by the total number $N$ of experiments, to obtain the probabilities $p_{\uparrow} = \frac{N_{\uparrow}}{N}, p_{\downarrow} = \frac{N_{\downarrow}}{N}$. We associate $\uparrow$ with eigenvalue $\lambda_{\uparrow} = 1$ and $\downarrow$ with eigenvalue $\lambda_{\downarrow} = -1$, and sum them, weighted by their probabilities. This is the expectation value $\langle \psi \mid Z \mid \psi \rangle = (1)p_{\uparrow} + (-1)p_{\uparrow}$. You could just imagine: we write down a $1$ or a $-1$ depending on the outcome, and them sum them all up and divide by $N$.

In [ ]:
def stochastic_expect(psi, n_shots=5000):
    dm = psi*psi.dag()
    return sum([np.random.choice([1, -1],\
                        p=[dm[0,0].real, dm[1,1].real])\
                    for i in range(n_shots)])/n_shots

psi = qt.rand_ket(2)
print(qt.expect(qt.sigmaz(), psi))
print(stochastic_expect(psi))

But hat about $\langle X \rangle$ or $\langle Y \rangle$?

Well, if you think about it, we just have to do a rotation before our measurement. Suppose our qubit is pointing up in the $Y^{+}$ direction, so a $Y$ measurement should give $\uparrow$ every time. But we're only allowed $Z$ measurements. We can get the same effect as a $Y$ measurement by first doing a rotation of $90^{\circ}$ aka $\frac{\pi}{2}$ around the $X$ axis, which rotates $Y^{+}$ into $Z^{+}$ (and $Y^{-}$ into $Z^{-}$). So we can turn a $Z$ measurement into a $Y$ measurement by first applying $R_{x}(\frac{\pi}{2}) = e^{-\frac{\pi}{4}iX}$ to $\mid \psi \rangle$, and then measuring.

Similarly, if we had to measure $\langle X \rangle$, we should apply $R_{y}(-\frac{\pi}{2}) = e^{\frac{\pi}{4}}iY$, rotating the opposite way a quarter turn around the $Y$ axis, to rotate $X^{+}$ into $Z^{+}$, etc.

In [ ]:
print(qt.expect(qt.sigmax(), psi))
print(stochastic_expect((1j*qt.sigmay()*np.pi/4).expm()*psi))
print()
print(qt.expect(qt.sigmay(), psi))
print(stochastic_expect((-1j*qt.sigmax()*np.pi/4).expm()*psi))

Putting it together, we decompose our matrix into a sum of Pauli's, and then for each term in this sum, we do a separate experiment (state preparation, pre-measurement rotation, measurement) many times, to accumulate the expectation value of that Pauli, which we weight in the overall sum, which gives us the expectation value of the original matrix on that state. 

In [ ]:
def expect(H, psi, basis):
    Rmap = {"I": qt.identity(2),\
            "X": (1j*qt.sigmay()*np.pi/4).expm(),\
            "Y": (-1j*qt.sigmax()*np.pi/4).expm(),\
            "Z": qt.identity(2)}
    H_pauli = op_pauli(H, basis)
    return sum([v*(stochastic_expect(Rmap[s]*psi)\
                       if s.count("I") != len(s)\
                   else 1)\
                if not np.isclose(v, 0) else 0
                    for s, v in H_pauli.items()])

print(qt.expect(H, psi))
print(expect(H, psi, IXYZ))

<hr>

So all that's great, but what if we have multiple qubits, say $n$?

Well, it turns out that strings of Paulis of length $n$ also form a basis for Hermitian matrices, in this case $2^{n} \times 2^{n}$. By a "string," I mean, a tensor product of Paulis, e.g.:

$XYYZIIIX$ means: $X \otimes Y \otimes Y \otimes Z \otimes I \otimes I \otimes I \otimes X$.

For two qubits, we'd have the 16 matrices:

$\begin{matrix} II & IX & IY & IZ \\
                XI & XX & XY & XZ \\
                YI & YX & YY & YZ \\
                ZI & ZX & XY & ZZ \end{matrix}$

So we can decompose an $n$-qubit operator just as easily as before:

In [ ]:
from itertools import product

def op_pauli(O, basis):
    return dict([(s, (o.dag()*O).tr()/np.sqrt(len(basis)))\
                    for s, o in basis.items()])

def pauli_op(P, basis):
    return sum([P[s]*o for s, o in basis.items()])

def construct_pauli_basis(n):
    IXYZ = {"I": qt.identity(2),\
            "X": qt.sigmax(),\
            "Y": qt.sigmay(),\
            "Z": qt.sigmaz()}
    return dict([("".join(p),\
                  qt.tensor(*[IXYZ[s] for s in p]))\
                for p in product(IXYZ.keys(), repeat=n)])

n = 2
basis = construct_pauli_basis(n)
#H = qt.rand_herm(2**n)
H = qt.Qobj(np.array([[1,0,0,0],\
                      [0,0,-1,0],\
                      [0,-1,0,0],\
                      [0,0,0,1]]))
H.dims = [[2]*n, [2]*n]
H_pauli = op_pauli(H, basis)
H2 = pauli_op(H_pauli, basis)

print(H)
print_pauli(H_pauli)
print(H2)

For example, we could decompose the following two qubit operator:

$H = \begin{pmatrix}1 & 0 & 0 & 0\\ 
0 & 0 & -1 & 0\\ 
0 & -1 & 0 & 0\\ 
0 & 0 & 0 & 1\end{pmatrix} = \frac{1}{2} -\frac{1}{2}XX -\frac{1}{2}YY + \frac{1}{2}ZZ $

The expectation value becomes:

$\langle H \rangle = \frac{1}{2} -\frac{1}{2}\langle XX \rangle -\frac{1}{2} \langle YY \rangle + \frac{1}{2} \langle ZZ \rangle$

<hr>

Now it is a fact that $A \otimes B \mid \alpha \rangle \mid \beta \rangle = A\mid \alpha \rangle \otimes B \mid \beta \rangle$ for unitaries $A$ and $B$. In other words, $A \otimes B$ on a separable state just transforms $\mid \alpha \rangle$ $A$ and $\mid \beta \rangle$ by $B$. One thing that this implies is that tensor products of the eigenvectors of $A$ and $B$ are also eigenvectors of $A \otimes B$, and that the corresponding eigenvalues of $A \otimes B$ are given by the products of $A$ and $B$ eigenvalues.


In [ ]:
XY = qt.tensor(qt.sigmax(), qt.sigmay())
Xl, Xv = qt.sigmax().eigenstates()
Yl, Yv = qt.sigmay().eigenstates()

l, v = [], []
for i in range(2):
    for j in range(2):
        l.append(Xl[i]*Yl[j])
        v.append(qt.tensor(Xv[i],Yv[j]))

print(l)
for v_ in v:
    print(v_)
    print(XY*v_)
    print()

You might notice however:

In [ ]:
XY.eigenstates()

gives different eigenstates!

Yet:

In [ ]:
XYl, XYv = XY.eigenstates()
print(XYl)
for v_ in v:
    print(v_)
    print(v_.transform(XYv))
    print()

We can see that each of the tensor product eigenstates, in the basis provided by the eigenstates calculated by qutip, lives either in the $1$ or $-1$ sector: in other words, can be written as a sum of eigenstates with the same eigenvalue, indeed, the same as the one corresponding to the product of eigenvalues of the original tensored eigenstates. So the ambiguity is due to the degeneracy of the eigenvalues. In any case, we're good to go!

<hr>


So for each Pauli string in the decomposition of our operator, e.g., $XYY$, we want to follow the same rule as before, just on each of the qubits. Recall:

$X \rightarrow R_{y}(-\frac{\pi}{2}) = e^{\frac{\pi}{4}}iY$

$Y \rightarrow R_{x}(\frac{\pi}{2}) = e^{-\frac{\pi}{4}iX}$

So for $XYY$,  we do a $Z$ measurement on each of the qubits in turn, but first: a rotation $R_{y}(-\frac{\pi}{2})$ on the first qubit, and $R_{x}(\frac{\pi}{2})$ on the second two.

Upon measurement, we get three bits, e.g., $1, -1, 1$, representing the results of a $Z$ measurement on each of the three qubits. We multiply these three eigenvalues together, to get the eigenvalue of the whole state: $-1$. We add it to the pile, and then repeat the experiment thousands of times, eventually dividing by the number of experiments. And then we have the expectation value of that Pauli operator, which then is weighted in the sum which gives the expectation value of the original operator.

In [ ]:
def upgrade(op, i, n):
    return qt.tensor(*[op if i == j \
                          else qt.identity(op.shape[0])\
                                for j in range(n)])

def expect_allZ(qubits, n_shots=1000, stochastic=True):
    n = len(qubits.dims[0])
    if not stochastic:
        return qt.expect(qt.tensor(*[qt.sigmaz()]*n), qubits)
    Zl, Zv = qt.sigmaz().eigenstates()
    Zp = [v*v.dag() for v in Zv]
    running = 0
    for i in range(n_shots):
        choices = []
        state = qubits.copy()
        for j in range(n):
            dm = state.ptrace(j)
            choice = np.random.choice([1, -1], p=[dm[0,0].real, dm[1,1].real])
            choices.append(choice)
            state = (upgrade(Zp[0 if choice == -1 else 1], j, n)*state).unit()
        running += np.prod(choices)
    running = running/n_shots
    return running

def expect(H, psi, basis, stochastic=True):
    Rmap = {"I": qt.identity(2),\
            "X": (1j*qt.sigmay()*np.pi/4).expm(),\
            "Y": (-1j*qt.sigmax()*np.pi/4).expm(),\
            "Z": qt.identity(2)}
    H_pauli = op_pauli(H, basis)
    return sum([v*(expect_allZ(\
                            qt.tensor(*[Rmap[o] for o in s])*psi,\
                            stochastic=stochastic)\
                               if s.count("I") != len(s)\
                   else 1)\
                if not np.isclose(v, 0) else 0
                    for s, v in H_pauli.items()])

psi = qt.rand_ket(4)
psi.dims = [[2,2],[1,1]]
print(qt.expect(H, psi))
print(expect(H, psi, basis))

So we can take expectation values to our heart's content. Now we turn to issue of parameterizing the state.

As before, we're working with:

$H = \begin{pmatrix}1 & 0 & 0 & 0\\ 
0 & 0 & -1 & 0\\ 
0 & -1 & 0 & 0\\ 
0 & 0 & 0 & 1\end{pmatrix} = \frac{1}{2} -\frac{1}{2}XX -\frac{1}{2}YY + \frac{1}{2}ZZ $

We'll take as our ansatz for the state: 

$\mid \psi(\theta) \rangle = (R_{x}(\theta) \otimes I)(CNOT)(h \otimes I)\mid \uparrow \uparrow \rangle$

Here, $h = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}$, the Hadamard gate, aka the Fourier Transform operator ($h^{\dagger}Xh = Z$).

$CNOT = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix}$ is the controlled-$X$ or controlled-$NOT$ gate, which flips the second qubit if the first qubit is $\mid \downarrow \rangle$. Acting on $\mid \uparrow \uparrow \rangle$, $h$ takes the first qubit to an equal superposition of $\uparrow$ and $\downarrow$: in other words, from $Z^{+}$ to $X^{+}$. The the $CNOT$ makes the flippage of the second qubit dependent on the first qubit which is in a superposition. This leads to the maximally entangled state: $\frac{1}{\sqrt{2}} (\mid \uparrow \uparrow \rangle + \mid \downarrow \downarrow \rangle)$. Finally, we perform a rotation of the first qubit around the $X$ axis by some angle $\theta$, which allows us to parameterize all manner of maximally entangled states, from correlated to anticorrelated.

We can then iterate over values of $\theta$ to try and find our ground state.

Putting it all together:


In [ ]:
%matplotlib inline

import numpy as np
import qutip as qt
from qutip.qip.operations import cnot, hadamard_transform
from itertools import product
import matplotlib.pyplot as plt

##############################################################################

def upgrade(op, i, n):
    return qt.tensor(*[op if i == j \
                          else qt.identity(op.shape[0])\
                                for j in range(n)])

##############################################################################

def construct_pauli_basis(n):
    IXYZ = {"I": qt.identity(2),\
            "X": qt.sigmax(),\
            "Y": qt.sigmay(),\
            "Z": qt.sigmaz()}
    return dict([("".join(p),\
                  qt.tensor(*[IXYZ[s] for s in p]))\
                for p in product(IXYZ.keys(), repeat=n)])
def op_pauli(O, basis):
    return dict([(s, (o.dag()*O).tr()/np.sqrt(len(basis)))\
                    for s, o in basis.items()])

def pauli_op(P, basis):
    return sum([P[s]*o for s, o in basis.items()])


def print_pauli(P):
    for s, v in P.items():
        if not np.isclose(v, 0):
            print("%s: %.2f" % (s, v))

##############################################################################

def expect_allZ(qubits, n_shots=1000, stochastic=True):
    n = len(qubits.dims[0])
    if not stochastic:
        return qt.expect(qt.tensor(*[qt.sigmaz()]*n), qubits)
    Zl, Zv = qt.sigmaz().eigenstates()
    Zp = [v*v.dag() for v in Zv]
    running = 0
    for i in range(n_shots):
        choices = []
        state = qubits.copy()
        for j in range(n):
            dm = state.ptrace(j)
            choice = np.random.choice([1, -1], p=[dm[0,0].real, dm[1,1].real])
            choices.append(choice)
            state = (upgrade(Zp[0 if choice == -1 else 1], j, n)*state).unit()
        running += np.prod(choices)
    running = running/n_shots
    return running

def expect(H, psi, basis, stochastic=True):
    Rmap = {"I": qt.identity(2),\
            "X": (1j*qt.sigmay()*np.pi/4).expm(),\
            "Y": (-1j*qt.sigmax()*np.pi/4).expm(),\
            "Z": qt.identity(2)}
    H_pauli = op_pauli(H, basis)
    return sum([v*(expect_allZ(\
                            qt.tensor(*[Rmap[o] for o in s])*psi,\
                            stochastic=stochastic)\
                               if s.count("I") != len(s)\
                   else 1)\
                if not np.isclose(v, 0) else 0
                    for s, v in H_pauli.items()])

##############################################################################

def vqe(H, ansatz, samples=100, stochastic=False, verbose=False, plot=False):
    n = len(H.dims[0])
    basis = construct_pauli_basis(n)
    expectation_values = []
    thetas = np.linspace(0, 2*np.pi, samples)
    for theta in thetas:
        expectation_values.append(expect(H, ansatz(theta), basis, stochastic=stochastic))
        print("theta = %.4f, <H> = %.4f" % (theta, expectation_values[-1]))\
            if verbose else None
    least_index = np.argmin(expectation_values)
    
    final_theta = thetas[least_index]
    final_expectation_value = expectation_values[least_index].real
    final_vector = ansatz(final_theta)
    print("final theta = %.4f, <H> = %.4f" % \
              (final_theta, final_expectation_value)) \
        if verbose else None
    print("final vector =\n%s" % ansatz(final_theta))\
        if verbose else None
    if plot:
        fig = plt.figure(figsize=(8,6))
        ax = fig.add_subplot(111)
        ax.plot(thetas, expectation_values)
        ax.set_xlabel('θ', fontsize=14)
        ax.set_ylabel('<H>', fontsize=14)

    return (final_theta, final_expectation_value, final_vector)

##############################################################################

def entangled_ansatz(theta):
    vac = qt.basis(4,0)
    vac.dims = [[2,2], [1,1]]
    return qt.tensor((-1j*qt.sigmax()*theta/2).expm(), qt.identity(2))*\
           cnot()*\
           qt.tensor(hadamard_transform(), qt.identity(2))*\
           vac

n = 2
basis = construct_pauli_basis(n)
H = qt.Qobj(np.array([[1,0,0,0],\
                      [0,0,-1,0],\
                      [0,-1,0,0],\
                      [0,0,0,1]]))
H.dims = [[2]*n, [2]*n]

##############################################################################

Hl, Hv = H.eigenstates()
print("compare:")
print("lowest eigval =  %.4f" % (Hl[0]))
print("eigenvector = \n%s" % (Hv[0]))
print()

final_theta, final_expectation_value, final_vector = \
    vqe(H, entangled_ansatz, samples=20, verbose=True, stochastic=False, plot=True)

<hr>

That's all well and good, but let's actually run this baby on a quantum computer.

We use IBM's framekwork qiskit. The algorithm directly translates over, although we form all the circuits ahead of time, transpile/simplify themand then send them in bulk to the quantum computer (or classical simulator), divvied up into several "jobs."

In [ ]:
%matplotlib inline

import numpy as np
import qutip as qt
from math import pi
from itertools import *
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, execute
from qiskit import Aer, IBMQ, transpile
from qiskit.providers.ibmq.managed import IBMQJobManager

##############################################################################

def construct_pauli_basis(n):
    IXYZ = {"I": qt.identity(2),\
            "X": qt.sigmax(),\
            "Y": qt.sigmay(),\
            "Z": qt.sigmaz()}
    return dict([("".join(p),\
                  qt.tensor(*[IXYZ[s] for s in p]))\
                for p in product(IXYZ.keys(), repeat=n)])
def op_pauli(O, basis):
    return dict([(s, (o.dag()*O).tr()/np.sqrt(len(basis)))\
                    for s, o in basis.items()])

def pauli_op(P, basis):
    return sum([P[s]*o for s, o in basis.items()])


def print_pauli(P):
    for s, v in P.items():
        if not np.isclose(v, 0):
            print("%s: %.2f" % (s, v))

##############################################################################

H = qt.Qobj(np.array([[1,0,0,0],\
                      [0,0,-1,0],\
                      [0,-1,0,0],\
                      [0,0,0,1]]))
H.dims = [[2]*2, [2]*2]
Hl, Hv = H.eigenstates()
    
print("compare:")
print("lowest eigval =  %.4f" % (Hl[0]))
print("eigenvector = \n%s" % (Hv[0]))
print()

basis = construct_pauli_basis(2)
H_pauli = op_pauli(H, basis)

##############################################################################

def ansatz(theta):
    circ = QuantumCircuit(2)
    circ.h(0)
    circ.cx(0,1)
    circ.rx(theta, 0)
    return circ

##############################################################################

local = False
if local:
    backend = Aer.get_backend('qasm_simulator') # local
else:    
    provider = IBMQ.load_account()
    job_manager = IBMQJobManager()

    #backend = provider.get_backend('ibmq_qasm_simulator')
    backend = provider.get_backend('ibmq_vigo')
    #backend = provider.get_backend('ibmq_5_yorktown')
    #backend = provider.get_backend('ibmq_16_melbourne')

samples = 30
n_shots = 4*1024

circs = []
circ_indices = []
thetas = np.linspace(0, 2*np.pi, samples)
for t in thetas:
    for n, v in H_pauli.items():
        if not np.isclose(v, 0):
            circ = ansatz(t)
            for i, o in enumerate(n):
                if o == "X":
                    circ.ry(-pi/2, i)
                elif o == "Y":
                    circ.rx(pi/2, i)
            circ.measure_all()
            circs.append(circ)
            circ_indices.append((t, n))
circs = transpile(circs, backend=backend)
jb = execute(circs, backend, shots=n_shots) if local \
        else job_manager.run(circs, backend=backend, name='vqe', shots=n_shots) 
jb_results = jb.result() if local else jb.results()
expectation_values = []
for t in thetas:
    values = []
    for n, v in H_pauli.items():
        if not np.isclose(v, 0):
            if n.count("I") == len(n):
                values.append(H_pauli[n])
            else:
                counts = jb_results.get_counts(circ_indices.index((t, n)))
                values.append(\
                    H_pauli[n]*\
                        sum([(c/n_shots)*\
                                np.prod([1 if s_=='1' \
                                            else -1 for s_ in s])\
                        for s, c in counts.items()])\
                    )
    e = sum(values)
    print("theta = %.4f, <H> = %.8f" % (t, e))
    expectation_values.append(e)
min_result = np.argmin(expectation_values)
final_theta = thetas[min_result]
print("final_theta: %.4f" % final_theta)
print("<H> = %.4f" % expectation_values[min_result])

state_backend = Aer.get_backend('statevector_simulator')
state = execute(ansatz(final_theta), backend=state_backend).result().get_statevector()
print("final vector = \n%s" % qt.Qobj(state))

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.plot(thetas, expectation_values)
ax.set_xlabel('θ', fontsize=14)
ax.set_ylabel('<H>', fontsize=14)

We can check out the transpiled circuits:

In [ ]:
[circ.draw() for circ in circs[:10]]

<hr>

So in the end, the answer was when $\theta = \pi$, the ansatz is $-\frac{i}{\sqrt{2}} \begin{pmatrix} 0 \\ 1 \\ 1 \\ 0 \end{pmatrix}$, which corresponds (up to phase) to the lowest eigenstate of $H$ with eigenvalue $-1$. We could imagine more sophisticated schemes, involving more elaborate ansatze and using fancier algorithms to search through the parameters, from gradient descent onward, generating thousands upon thousands of circuits that when implemented thousands and thousands of time allow one to estimate the lowest eigenstate of the operator faster than a classical computer.

Special thanks to <a href="https://qosf.org/">qosf</a>, and <a href="https://www.mustythoughts.com/variational-quantum-eigensolver-explained">this</a>.